In [1]:
import numpy as np
from PIL import Image
from numpy import asarray
from matplotlib import image
from scipy.linalg import lu, qr, solve
from scipy.linalg import lu_factor, lu_solve
from matplotlib import pyplot as plt
import math
import scipy.linalg
import time
from typing import Tuple, Union
from typing import Union
from scipy.sparse.linalg import LinearOperator, lsqr
from scipy.optimize import least_squares
from scipy.optimize import minimize
from scipy.linalg import svd
from skimage.metrics import peak_signal_noise_ratio as psnr

In [30]:
def cal_PSNR(X, X_trunc):
    try:
        m, n, t = X.shape
    except:
        m, n = X.shape
        t=1
    X_flat = X.reshape(-1, X.shape[-1])
    X_trunc_flat = X_trunc.reshape(-1, X_trunc.shape[-1])
    i = np.linalg.norm(X_flat - X_trunc_flat, "fro") ** 2
    return 10 * (math.log10((n**2) / i))

In [31]:
def load_image(name):
    im = Image.open("/Users/yangjinglan/Desktop/DDA3005/project/DDA3005_Project_Jinglan_Yang/test_images/"+name+"_original.png") 
    img = np.array(im)
    img = img.astype(np.float64) / 255

    try:
        m, n, t = img.shape
    except:
        m, n = img.shape
        t=1
    print("m=",m,"n=",n,"t=",t)
    img[42,0:n-1] = 0
    return img,m,n,t

In [32]:
def create_kernel(n, j, k):
    kernel = np.zeros((n, n))
    for t in range(0, n):
        for r in range(0, n):
            i = t - r + n
            if (i >= n + j - k + 1) and (i <= n + j):
                kernel[t, r] = (2 / (k * (k + 1))) * (k + i - n - j)
    return kernel

In [33]:
def lu_inverse(A):
    P, L, U = scipy.linalg.lu(A)
    if not np.all(np.isfinite(U.diagonal())):
        raise ValueError("Matrix is not invertible.")
    U_inv = np.linalg.inv(U)
    #U_inv = np.linalg.pinv(U)
    L_inv = L.T

    A_inv = np.dot(U_inv, L_inv)

    return A_inv,L_inv, U_inv

In [51]:
def qr_inverse(A):
    Q, R = np.linalg.qr(A)
    if not np.all(np.isfinite(R.diagonal())):
        raise ValueError("Matrix is not invertible.")
    #R_inv = np.linalg.inv(R)
    R_inv = np.linalg.pinv(R) # R may be singular, so use pinv() instead of inv()
    Q_inv = Q.T

    A_inv = np.dot(R_inv, Q_inv)

    return A_inv, Q_inv, R_inv

In [35]:
def householder_vectorized(a):
    v = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))
    v[0] = 1
    tau = 2 / (v.T @ v)
    return v,tau

def qr_decomposition(A):
    m,n = A.shape
    R = A.copy()
    Q = np.identity(m)
    
    for j in range(0, n):
        v, tau = householder_vectorized(R[j:, j, np.newaxis])        
        H = np.identity(m)
        H[j:, j:] -= tau * (v @ v.T)
        R = H @ R
        Q = H @ Q
        
    return Q[:n].T, np.triu(R[:n])

def HH_qr_inverse(A):
    Q, R = qr_decomposition(A)
    if not np.all(np.isfinite(R.diagonal())):
        raise ValueError("Matrix is not invertible.")
    #R_inv = np.linalg.inv(R)
    R_inv = np.linalg.pinv(R)
    Q_inv = Q.T
    #A_inv= np.dot(R_inv, Q_inv)
    return Q_inv, R_inv

In [36]:
def blurring(X, A_l, A_r):
    try:
        m, n, t = X.shape
    except:
        m, n = X.shape
        t=1  

    if t>1:
        B_final = np.zeros((m,n,t))
    else:
        B_final = np.zeros((m,n))

    if t>1:
        for i in range(t):
            X_p = np.squeeze(X[:, :, i : i + 1])
            B = np.dot(np.dot(A_l, X_p), A_r)
            B_final[:, :, i : i + 1] = B[:, :, np.newaxis]  
    else:
        X_p=X
        B = np.dot(np.dot(A_l, X_p), A_r)
        B_final=B

    return B_final

In [37]:
def construct_A(n, j_l, k_l, j_r, k_r):
    A_l = create_kernel(n, j_l, k_l)
    A_r = create_kernel(n, j_r, k_r)

    # Compute determinant
    det_A_l = np.linalg.det(A_l)
    print(f"Determinant of A_l: {det_A_l}")
    # Check singularity
    if np.isclose(det_A_l, 0):
        print("Matrix A_l is singular.")
    else:
        print("Matrix A_l is non-singular.")

    # Compute determinant
    det_A_r = np.linalg.det(A_r)
    print(f"Determinant of A_r: {det_A_r}")
    # Check singularity
    if np.isclose(det_A_r, 0):
        print("Matrix A_r is singular.")
    else:
        print("Matrix A_r is non-singular.")
    return A_l, A_r

In [38]:
def LU_deblurring(B,A_l,A_r):
    try:
        m, n, t = B.shape
    except:
        m, n = B.shape
        t=1
    A_l_inv_lu, L_l_inv, U_l_inv=lu_inverse(A_l)
    A_r_inv_lu, L_r_inv, U_r_inv=lu_inverse(A_r)
    if t>1:
        X_final_lu = np.zeros((m,n,t))
        for i in range(t):
            B_p = np.squeeze(B[:, :, i : i + 1])

            X1=np.dot(U_r_inv, L_r_inv)
            X2=np.dot(B_p,X1)
            X3=np.dot(L_l_inv,X2)
            X_lu=np.dot(U_l_inv,X3)

            X_final_lu[:, :, i : i + 1] = X_lu[:, :, np.newaxis]
    else:
        X_final_lu = np.zeros((m,n))
        B_p = B

        X1=np.dot(U_r_inv, L_r_inv)
        X2=np.dot(B_p,X1)
        X3=np.dot(L_l_inv,X2)
        X_lu=np.dot(U_l_inv,X3)
        X_final_lu=X_lu
    return X_final_lu


In [39]:
def QR_deblurring(B,A_l,A_r):
    try:
        m, n, t = B.shape
    except:
        m, n = B.shape
        t=1

    A_l_inv, Q_l_inv, R_l_inv=qr_inverse(A_l)
    A_r_inv,  Q_r_inv, R_r_inv=qr_inverse(A_r)

    if t>1:
        X_final_qr = np.zeros((m,n,t))
        for i in range(t):
            B_p = np.squeeze(B[:, :, i : i + 1])
            Y1=np.dot(R_r_inv, Q_r_inv)
            Y2=np.dot(B_p,Y1)
            Y3=np.dot(R_l_inv,Y2)
            X_qr=np.dot(Q_l_inv,Y3)
            X_final_qr[:, :, i : i + 1] = X_qr[:, :, np.newaxis]
    else:
        X_final_qr = np.zeros((m,n))
        B_p = B
        Y1=np.dot(R_r_inv, Q_r_inv)
        Y2=np.dot(B_p,Y1)
        Y3=np.dot(R_l_inv,Y2)
        X_qr=np.dot(Q_l_inv,Y3)
        X_final_qr=X_qr
    return X_final_qr


In [40]:
def Householder_QR_deblurring(B,A_l,A_r):
    try:
        m, n, t = B.shape
    except:
        m, n = B.shape
        t=1

    Q_l_inv_H, R_l_inv_H=HH_qr_inverse(A_l)
    Q_r_inv_H, R_r_inv_H=HH_qr_inverse(A_r)

    if t>1:
        X_final_qr = np.zeros((m,n,t))
        for i in range(t):
            B_p = np.squeeze(B[:, :, i : i + 1])
            Y1=np.dot(R_r_inv_H, Q_r_inv_H)
            Y2=np.dot(B_p,Y1)
            Y3=np.dot(R_l_inv_H,Y2)
            X_qr=np.dot(Q_l_inv_H,Y3)
            X_final_qr[:, :, i : i + 1] = X_qr[:, :, np.newaxis]
    else:
        X_final_qr = np.zeros((m,n))
        B_p = B
        Y1=np.dot(R_r_inv_H, Q_r_inv_H)
        Y2=np.dot(B_p,Y1)
        Y3=np.dot(R_l_inv_H,Y2)
        X_qr=np.dot(Q_l_inv_H,Y3)
        X_final_qr=X_qr
    return X_final_qr

In [41]:
def least_squares_deblur_regularized(B, A_left, A_right, reg_param=1e-6, init=None, max_iter=1000, tol=1e-12):
    n = B.shape[0]

    # Objective function with regularization
    def objective(X_vec):
        X = X_vec.reshape(n, n)
        residual = A_left @ X @ A_right - B
        # Add the regularization term to the objective
        return np.linalg.norm(residual, 'fro')**2 + reg_param * np.linalg.norm(X, 'fro')**2

    # Gradient of the objective function with regularization
    def gradient(X_vec):
        X = X_vec.reshape(n, n)
        residual = A_left @ X @ A_right - B
        grad_residual = 2 * (A_left.T @ residual @ A_right.T)
        grad_regularization = 2 * reg_param * X
        return (grad_residual + grad_regularization).flatten()

    # Initial guess for optimization
    if init is None:
        X0 = np.zeros_like(B).flatten()  # Default: Zero initialization
    else:
        X0 = init.flatten()

    # Perform optimization
    result = minimize(
        objective, 
        X0, 
        jac=gradient, 
        method='L-BFGS-B', 
        options={'maxiter': max_iter, 'disp': False, 'ftol': tol}
    )

    # Reshape result to matrix form
    return result.x.reshape(n, n)


In [42]:
def LS_deblurring(B,A_l,A_r, X_qr):
    try:
        m, n, t = B.shape
    except:
        m, n = B.shape
        t=1

    if t>1:
        X_final_ls = np.zeros((m,n,t))
        for i in range(t):
            B_p = np.squeeze(B[:, :, i : i + 1])
            if not isinstance(X_qr, type(None)):
                x0 = np.squeeze(X_qr[:, :, i : i + 1])
            else:
                x0=None
            X_ls = least_squares_deblur_regularized(B_p,A_l, A_r, init=x0)
            X_final_ls[:, :, i : i + 1] = X_ls[:, :, np.newaxis]
    else:
        X_final_ls = np.zeros((m,n))
        B_p = B
        x0=X_qr
        X_ls = least_squares_deblur_regularized(B_p,A_l, A_r, init=x0)
        X_final_ls= X_ls
    return X_final_ls

In [43]:
# Precondition matrices
def precondition_matrix(A):
    U, S, Vt = svd(A, full_matrices=False)
    S_inv = np.diag(1 / (S + 1e-5))  # Add small regularization
    return U @ S_inv @ Vt

In [44]:
def save_image(matrix, name, filepath):
    """Save a matrix as an image."""
    matrix = (matrix * 255).astype(np.uint8)
    image = Image.fromarray(matrix)
    if image.mode == "RGBA":
        image = image.convert("RGB")
    image.save(filepath+name+".png")

In [50]:
###
# LU factorization + QR factorization
# direct inverse
###
image_path=["256_buildings","1250_m3"]
output_path="/Users/yangjinglan/Desktop/DDA3005/project/DDA3005_Project_Jinglan_Yang/test_images/"
j_l=0
j_r=1
k_l=12
k_r1=24
k_r2=36
k_r_list=[24,36]
for image in image_path:
    for k_r in k_r_list:
        print("image:",image)
        print("k_r=",k_r)
        img,m,n,t=load_image(image)
        A_l,A_r=construct_A(n,j_l, k_l, j_r, k_r)
        B=blurring(img,A_l,A_r)
        save_image(B,image+"_blurred_"+str(k_r),output_path)
        #LU
        start_time = time.time()
        X_lu=LU_deblurring(B,A_l,A_r)
        end_time = time.time()
        execution_time_LU = end_time - start_time
        save_image(X_lu,image+"_lu_"+str(k_r),output_path)
        print("cpu-time of LU: ",execution_time_LU)
        PSNR_lu=cal_PSNR(img,X_lu)
        psnr_value = psnr(img, X_lu, data_range=1.0)
        print("PSNR_lu=",PSNR_lu)
        print(f"PSNR_lu: {psnr_value} dB")
        #QR
        start_time = time.time()
        X_qr=QR_deblurring(B,A_l,A_r)
        end_time = time.time()
        execution_time_QR = end_time - start_time
        save_image(X_qr,image+"_qr_"+str(k_r),output_path)
        print("cpu-time of QR: ",execution_time_QR)
        PSNR_qr=cal_PSNR(img,X_qr)
        psnr_value = psnr(img, X_qr, data_range=1.0)
        print("PSNR_qr=",PSNR_qr)
        print(f"PSNR_qr: {psnr_value} dB")

image: 256_buildings
k_r= 24
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -5.719118249267804e-293
Matrix A_r is singular.
cpu-time of LU:  0.05022621154785156
PSNR_lu= 21.562335166332236
PSNR_lu: 21.562335166332236 dB
cpu-time of QR:  0.03811001777648926
PSNR_qr= 75.76400872645509
PSNR_qr: 75.76400872645509 dB
image: 256_buildings
k_r= 36
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -0.0
Matrix A_r is singular.
cpu-time of LU:  0.02767205238342285
PSNR_lu= 21.43361561842454
PSNR_lu: 21.43361561842454 dB
cpu-time of QR:  0.04233384132385254
PSNR_qr= 131.15737486568943
PSNR_qr: 131.1573748656894 dB
image: 1250_m3
k_r= 24
m= 1250 n= 1250 t= 4
Determinant of A_l: 0.0
Matrix A_l is singular.
Determinant of A_r: 0.0
Matrix A_r is singular.
cpu-time of LU:  1.2650511264801025
PSNR_lu= -655.0928494360734
PSNR_lu: -649.0722495227938 dB


LinAlgError: Singular matrix

In [17]:
###
# LU factorization + QR factorization
# pseudo-inverse
###
image_path=["256_buildings","1250_m3"]
output_path="/Users/yangjinglan/Desktop/DDA3005/project/DDA3005_Project_Jinglan_Yang/test_images/"
j_l=0
j_r=1
k_l=12
k_r1=24
k_r2=36
k_r_list=[24,36]
for image in image_path:
    for k_r in k_r_list:
        print("image:",image)
        print("k_r=",k_r)
        img,m,n,t=load_image(image)
        A_l,A_r=construct_A(n,j_l, k_l, j_r, k_r)
        B=blurring(img,A_l,A_r)
        save_image(B,image+"_blurred_"+str(k_r),output_path)
        #LU
        start_time = time.time()
        X_lu=LU_deblurring(B,A_l,A_r)
        end_time = time.time()
        execution_time_LU = end_time - start_time
        save_image(X_lu,image+"_lu_"+str(k_r),output_path)
        print("cpu-time of LU: ",execution_time_LU)
        PSNR_lu=cal_PSNR(img,X_lu)
        psnr_value = psnr(img, X_lu, data_range=1.0)
        print("PSNR_lu=",PSNR_lu)
        print(f"PSNR_lu: {psnr_value} dB")
        #QR
        start_time = time.time()
        X_qr=QR_deblurring(B,A_l,A_r)
        end_time = time.time()
        execution_time_QR = end_time - start_time
        save_image(X_qr,image+"_qr_"+str(k_r),output_path)
        print("cpu-time of QR: ",execution_time_QR)
        PSNR_qr=cal_PSNR(img,X_qr)
        psnr_value = psnr(img, X_qr, data_range=1.0)
        print("PSNR_qr=",PSNR_qr)
        print(f"PSNR_qr: {psnr_value} dB")

image: 256_buildings
k_r= 24
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -5.719118249267804e-293
Matrix A_r is singular.
cpu-time of LU:  0.9948790073394775
PSNR_lu= 21.56178144260659
PSNR_lu: 21.56178144260659 dB
cpu-time of QR:  0.12429594993591309
PSNR_qr= 75.48131173239712
PSNR_qr: 75.48131173239712 dB
image: 256_buildings
k_r= 36
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -0.0
Matrix A_r is singular.
cpu-time of LU:  0.10610222816467285
PSNR_lu= 21.433616766972246
PSNR_lu: 21.433616766972246 dB
cpu-time of QR:  0.12587714195251465
PSNR_qr= 131.22993373298743
PSNR_qr: 131.2299337329874 dB
image: 1250_m3
k_r= 24
m= 1250 n= 1250 t= 4
Determinant of A_l: 0.0
Matrix A_l is singular.
Determinant of A_r: 0.0
Matrix A_r is singular.
cpu-time of LU:  3.9033820629119873
PSNR_lu= 16.907661946818394
PSNR_lu: 22.928261860097663 dB
cpu-time of QR:  4.158468008041382
PSNR_

In [46]:
###
# Householder QR factorization from scratch
# direct inverse
###
image_path=["256_buildings","1250_m3"]
output_path="/Users/yangjinglan/Desktop/DDA3005/project/DDA3005_Project_Jinglan_Yang/test_images/"
j_l=0
j_r=1
k_l=12
k_r1=24
k_r2=36
k_r_list=[24,36]
for image in image_path:
    for k_r in k_r_list:
        print("image:",image)
        print("k_r=",k_r)
        img,m,n,t=load_image(image)
        A_l,A_r=construct_A(n,j_l, k_l, j_r, k_r)
        B=blurring(img,A_l,A_r)
        save_image(B,image+"_blurred_"+str(k_r),output_path)

        #householder QR
        start_time = time.time()
        X_HHqr=Householder_QR_deblurring(B,A_l,A_r)
        end_time = time.time()
        execution_time_HHQR = end_time - start_time
        save_image(X_HHqr,image+"_HHqr_"+str(k_r),output_path)
        print("cpu-time of householder QR: ",execution_time_HHQR)
        PSNR_HHqr=cal_PSNR(img,X_HHqr)
        psnr_value = psnr(img, X_HHqr, data_range=1.0)
        print("PSNR_Householderqr=",PSNR_HHqr)
        print(f"PSNR_Householderqr: {psnr_value} dB")

image: 256_buildings
k_r= 24
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -5.719118249267804e-293
Matrix A_r is singular.
cpu-time of householder QR:  0.9932899475097656
PSNR_Householderqr= 75.28739131783583
PSNR_Householderqr: 75.28739131783581 dB
image: 256_buildings
k_r= 36
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -0.0
Matrix A_r is singular.
cpu-time of householder QR:  0.9856157302856445
PSNR_Householderqr= 131.4664653972169
PSNR_Householderqr: 131.4664653972169 dB
image: 1250_m3
k_r= 24
m= 1250 n= 1250 t= 4
Determinant of A_l: 0.0
Matrix A_l is singular.
Determinant of A_r: 0.0
Matrix A_r is singular.


KeyboardInterrupt: 

In [18]:
###
# Householder QR factorization from scratch
# pseudo-inverse
###
image_path=["256_buildings","1250_m3"]
output_path="/Users/yangjinglan/Desktop/DDA3005/project/DDA3005_Project_Jinglan_Yang/test_images/"
j_l=0
j_r=1
k_l=12
k_r1=24
k_r2=36
k_r_list=[24,36]
for image in image_path:
    for k_r in k_r_list:
        print("image:",image)
        print("k_r=",k_r)
        img,m,n,t=load_image(image)
        A_l,A_r=construct_A(n,j_l, k_l, j_r, k_r)
        B=blurring(img,A_l,A_r)
        save_image(B,image+"_blurred_"+str(k_r),output_path)

        #householder QR
        start_time = time.time()
        X_HHqr=Householder_QR_deblurring(B,A_l,A_r)
        end_time = time.time()
        execution_time_HHQR = end_time - start_time
        save_image(X_HHqr,image+"_HHqr_"+str(k_r),output_path)
        print("cpu-time of householder QR: ",execution_time_HHQR)
        PSNR_HHqr=cal_PSNR(img,X_HHqr)
        psnr_value = psnr(img, X_HHqr, data_range=1.0)
        print("PSNR_Householderqr=",PSNR_HHqr)
        print(f"PSNR_Householderqr: {psnr_value} dB")

image: 256_buildings
k_r= 24
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -5.719118249267804e-293
Matrix A_r is singular.
cpu-time of householder QR:  1.107496976852417
PSNR_Householderqr= 75.41114601425282
PSNR_Householderqr: 75.41114601425282 dB
image: 256_buildings
k_r= 36
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -0.0
Matrix A_r is singular.
cpu-time of householder QR:  1.1380589008331299
PSNR_Householderqr= 131.33802544540882
PSNR_Householderqr: 131.33802544540882 dB
image: 1250_m3
k_r= 24
m= 1250 n= 1250 t= 4
Determinant of A_l: 0.0
Matrix A_l is singular.
Determinant of A_r: 0.0
Matrix A_r is singular.


<ipython-input-7-6034445ecd77>:2: RuntimeWarning: invalid value encountered in true_divide
  v = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))


cpu-time of householder QR:  303.6416001319885
PSNR_Householderqr= 49.713738059043166
PSNR_Householderqr: 55.73433797232273 dB
image: 1250_m3
k_r= 36
m= 1250 n= 1250 t= 4
Determinant of A_l: 0.0
Matrix A_l is singular.
Determinant of A_r: 0.0
Matrix A_r is singular.
cpu-time of householder QR:  302.4939818382263
PSNR_Householderqr= 53.17136065687926
PSNR_Householderqr: 59.191960570158756 dB


In [20]:
###
# least square with QR factorization results(pseudo-inverse) as initial start
###
image_path=["256_buildings","1250_m3"]
output_path="/Users/yangjinglan/Desktop/DDA3005/project/DDA3005_Project_Jinglan_Yang/test_images/"
j_l=0
j_r=1
k_l=12
k_r1=24
k_r2=36
k_r_list=[24,36]
for image in image_path:
    for k_r in k_r_list:
        print("image:",image)
        print("k_r=",k_r)
        img,m,n,t=load_image(image)
        A_l,A_r=construct_A(n,j_l, k_l, j_r, k_r)
        B=blurring(img,A_l,A_r)
        save_image(B,image+"_blurred_"+str(k_r),output_path)

        #QR
        start_time = time.time()
        X_qr=QR_deblurring(B,A_l,A_r)
        end_time = time.time()
        execution_time_QR = end_time - start_time
        save_image(X_qr,image+"_qr_"+str(k_r),output_path)
        print("cpu-time of QR: ",execution_time_QR)
        PSNR_qr=cal_PSNR(img,X_qr)
        psnr_value = psnr(img, X_qr, data_range=1.0)
        print("PSNR_qr=",PSNR_qr)
        print(f"PSNR_qr: {psnr_value} dB")
        
        #least square
        start_time = time.time()
        X_ls=LS_deblurring(B,A_l,A_r,X_qr)
        end_time = time.time()
        execution_time_ls = end_time - start_time
        save_image(X_ls,image+"_ls_"+str(k_r),output_path)
        print("cpu-time of ls: ",execution_time_ls)
        PSNR_ls=cal_PSNR(img,X_ls)
        psnr_value = psnr(img, X_ls, data_range=1.0)
        print("PSNR_ls=",PSNR_ls)
        print(f"PSNR_ls: {psnr_value} dB")

image: 256_buildings
k_r= 24
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -5.719118249267804e-293
Matrix A_r is singular.
cpu-time of QR:  0.11359524726867676
PSNR_qr= 75.48131173239712
PSNR_qr: 75.48131173239712 dB
cpu-time of ls:  0.09817814826965332
PSNR_ls= 75.49108807956597
PSNR_ls: 75.49108807956597 dB
image: 256_buildings
k_r= 36
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -0.0
Matrix A_r is singular.
cpu-time of QR:  0.15976190567016602
PSNR_qr= 131.22993373298743
PSNR_qr: 131.2299337329874 dB
cpu-time of ls:  0.05468487739562988
PSNR_ls= 131.22993373298743
PSNR_ls: 131.2299337329874 dB
image: 1250_m3
k_r= 24
m= 1250 n= 1250 t= 4
Determinant of A_l: 0.0
Matrix A_l is singular.
Determinant of A_r: 0.0
Matrix A_r is singular.
cpu-time of QR:  4.119282960891724
PSNR_qr= 49.71373805904305
PSNR_qr: 55.73433797232262 dB
cpu-time of ls:  7.668555974960327
PSNR_ls=

In [21]:
###
# least square with no initial start
###
image_path=["256_buildings","1250_m3"]
output_path="/Users/yangjinglan/Desktop/DDA3005/project/DDA3005_Project_Jinglan_Yang/test_images/"
j_l=0
j_r=1
k_l=12
k_r1=24
k_r2=36
k_r_list=[24,36]
for image in image_path:
    for k_r in k_r_list:
        print("image:",image)
        print("k_r=",k_r)
        img,m,n,t=load_image(image)
        A_l,A_r=construct_A(n,j_l, k_l, j_r, k_r)
        B=blurring(img,A_l,A_r)
        save_image(B,image+"_blurred_"+str(k_r),output_path)

        #least square    
        start_time = time.time()
        X_ls=LS_deblurring(B,A_l,A_r,None)
        end_time = time.time()
        execution_time_ls = end_time - start_time
        save_image(X_ls,image+"_ls_no_initial_"+str(k_r),output_path)
        print("cpu-time of ls: ",execution_time_ls)
        PSNR_ls=cal_PSNR(img,X_ls)
        psnr_value = psnr(img, X_ls, data_range=1.0)
        print("PSNR_ls(no initial)=",PSNR_ls)
        print(f"PSNR_ls(no initial): {psnr_value} dB")

image: 256_buildings
k_r= 24
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -5.719118249267804e-293
Matrix A_r is singular.
cpu-time of ls:  2.6472792625427246
PSNR_ls(no initial)= 45.114895359528155
PSNR_ls(no initial): 45.114895359528155 dB
image: 256_buildings
k_r= 36
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -0.0
Matrix A_r is singular.
cpu-time of ls:  2.7475619316101074
PSNR_ls(no initial)= 41.552359453863666
PSNR_ls(no initial): 41.55235945386367 dB
image: 1250_m3
k_r= 24
m= 1250 n= 1250 t= 4
Determinant of A_l: 0.0
Matrix A_l is singular.
Determinant of A_r: 0.0
Matrix A_r is singular.
cpu-time of ls:  647.6934762001038
PSNR_ls(no initial)= 42.97388053144746
PSNR_ls(no initial): 48.9944804447271 dB
image: 1250_m3
k_r= 36
m= 1250 n= 1250 t= 4
Determinant of A_l: 0.0
Matrix A_l is singular.
Determinant of A_r: 0.0
Matrix A_r is singular.
cpu-time of ls:  742.

In [23]:
###
# Preconditioning A, least square with QR factorization (pseudo-inverse) results as initial start
###
image_path=["256_buildings","1250_m3"]
output_path="/Users/yangjinglan/Desktop/DDA3005/project/DDA3005_Project_Jinglan_Yang/test_images/"
j_l=0
j_r=1
k_l=12
k_r1=24
k_r2=36
k_r_list=[24,36]
for image in image_path:
    for k_r in k_r_list:
        print("image:",image)
        print("k_r=",k_r)
        img,m,n,t=load_image(image)
        A_l,A_r=construct_A(n,j_l, k_l, j_r, k_r)
        B=blurring(img,A_l,A_r)
        save_image(B,image+"_blurred_"+str(k_r),output_path)

        #QR
        start_time = time.time()
        X_qr=QR_deblurring(B,A_l,A_r)
        end_time = time.time()
        execution_time_QR = end_time - start_time
        save_image(X_qr,image+"_qr_"+str(k_r),output_path)
        print("cpu-time of QR: ",execution_time_QR)
        PSNR_qr=cal_PSNR(img,X_qr)
        psnr_value = psnr(img, X_qr, data_range=1.0)
        print("PSNR_qr=",PSNR_qr)
        print(f"PSNR_qr: {psnr_value} dB")        

        A_l_preconditioned = precondition_matrix(A_l)
        A_r_preconditioned = precondition_matrix(A_r)
        
        print("With preconditioning A:")
        start_time = time.time()
        X_ls=LS_deblurring(B,A_l_preconditioned,A_r_preconditioned,X_qr)
        end_time = time.time()
        execution_time_ls = end_time - start_time
        save_image(X_ls,image+"_ls_precondition_"+str(k_r),output_path)
        print("cpu-time of ls(preconditioning): ",execution_time_ls)
        PSNR_ls=cal_PSNR(img,X_ls)
        psnr_value = psnr(img, X_ls, data_range=1.0)
        print("PSNR_ls(preconditioning)=",PSNR_ls)
        print(f"PSNR_ls(preconditioning): {psnr_value} dB")

image: 256_buildings
k_r= 24
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -5.719118249267804e-293
Matrix A_r is singular.
cpu-time of QR:  0.1254279613494873
PSNR_qr= 75.48131173239712
PSNR_qr: 75.48131173239712 dB
With preconditioning A:
cpu-time of ls(preconditioning):  9.435670137405396
PSNR_ls(preconditioning)= 37.99486941383369
PSNR_ls(preconditioning): 37.99486941383369 dB
image: 256_buildings
k_r= 36
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -0.0
Matrix A_r is singular.
cpu-time of QR:  0.13127493858337402
PSNR_qr= 131.22993373298743
PSNR_qr: 131.2299337329874 dB
With preconditioning A:
cpu-time of ls(preconditioning):  12.921133995056152
PSNR_ls(preconditioning)= 41.70406273051191
PSNR_ls(preconditioning): 41.70406273051191 dB
image: 1250_m3
k_r= 24
m= 1250 n= 1250 t= 4
Determinant of A_l: 0.0
Matrix A_l is singular.
Determinant of A_r: 0.0
Matrix A_r is 

In [24]:
###
# Preconditioning A, least square with no initial start
###
image_path=["256_buildings","1250_m3"]
output_path="/Users/yangjinglan/Desktop/DDA3005/project/DDA3005_Project_Jinglan_Yang/test_images/"
j_l=0
j_r=1
k_l=12
k_r1=24
k_r2=36
k_r_list=[24,36]
for image in image_path:
    for k_r in k_r_list:
        print("image:",image)
        print("k_r=",k_r)
        img,m,n,t=load_image(image)
        A_l,A_r=construct_A(n,j_l, k_l, j_r, k_r)
        B=blurring(img,A_l,A_r)
        save_image(B,image+"_blurred_"+str(k_r),output_path)

        #least square     
        A_l_preconditioned = precondition_matrix(A_l)
        A_r_preconditioned = precondition_matrix(A_r)
        
        print("With preconditioning A:")

        start_time = time.time()
        X_ls=LS_deblurring(B,A_l_preconditioned,A_r_preconditioned,None)
        end_time = time.time()
        execution_time_ls = end_time - start_time
        save_image(X_ls,image+"_ls_precondition_noInitial_"+str(k_r),output_path)
        print("cpu-time of ls(no initial): ",execution_time_ls)
        PSNR_ls=cal_PSNR(img,X_ls)
        psnr_value = psnr(img, X_ls, data_range=1.0)
        print("PSNR_ls(no initial)=",PSNR_ls)
        print(f"PSNR_ls(no initial): {psnr_value} dB")

image: 256_buildings
k_r= 24
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -5.719118249267804e-293
Matrix A_r is singular.
With preconditioning A:
cpu-time of ls(no initial):  9.53705883026123
PSNR_ls(no initial)= 7.65166511044024
PSNR_ls(no initial): 7.65166511044024 dB
image: 256_buildings
k_r= 36
m= 256 n= 256 t= 1
Determinant of A_l: 7.837556767119042e-209
Matrix A_l is singular.
Determinant of A_r: -0.0
Matrix A_r is singular.
With preconditioning A:
cpu-time of ls(no initial):  9.33344316482544
PSNR_ls(no initial)= 7.439967825387296
PSNR_ls(no initial): 7.439967825387297 dB
image: 1250_m3
k_r= 24
m= 1250 n= 1250 t= 4
Determinant of A_l: 0.0
Matrix A_l is singular.
Determinant of A_r: 0.0
Matrix A_r is singular.
With preconditioning A:
cpu-time of ls(no initial):  1733.9094350337982
PSNR_ls(no initial)= -2.2374807946993522
PSNR_ls(no initial): 3.7831191185802133 dB
image: 1250_m3
k_r= 36
m= 1250 n= 1250 t= 4
Determinant o